# Get StrataScratch Subscribers with .subscribers() Youtube API

In [ ]:
!pip install google-auth
!pip install google-auth-oauthlib

In [2]:
import pandas as pd
import os
import pickle

from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request

import googleapiclient.discovery
import googleapiclient.errors

load credentials

In [29]:
credentials = None

# token.pickle stores the user's credentials from previously successful logins
if os.path.exists('token.pickle'):
    print('Loading Credentials From File...')
    with open('token.pickle', 'rb') as token:
        credentials = pickle.load(token)

# Google's Request
from google.auth.transport.requests import Request


# If there are no valid credentials available, then either refresh the token or log in.
if not credentials or not credentials.valid:
    if credentials and credentials.expired and credentials.refresh_token:
        print('Refreshing Access Token...')
        credentials.refresh(Request())
    else:
        print('Fetching New Tokens...')
        flow = InstalledAppFlow.from_client_secrets_file(
            'client_secrets.json',
            scopes=[
                'https://www.googleapis.com/auth/youtube.readonly'
            ]
        )

        flow.run_local_server(port=8080, prompt='consent')#,authorization_prompt_message='')
        credentials = flow.credentials

        # Save the credentials for the next run
        with open('token.pickle', 'wb') as f:
            print('Saving Credentials for Future Use...')
            pickle.dump(credentials, f)

Loading Credentials From File...
Refreshing Access Token...


In [30]:
#Get first response for sub/comms subscriptions that are allowed:
def get_subcomms_subscriptions_first_response(channelId):
        api_service_name = "youtube"
        api_version = "v3"

        youtube = googleapiclient.discovery.build(api_service_name, api_version, credentials=credentials)
        request = youtube.subscriptions().list(part="snippet",channelId=channelId)
        # request = youtube.subscriptions().list(part="snippet,contentDetails",pageToken='CLYHEAE', maxResults=maxResults,mySubscribers=True)
        response = request.execute()
        return response, request, youtube

In [31]:
def get_subscriber_ids(df,response):
    for sub in response['items']:
        if sub['kind'] == 'youtube#subscription': 
            
            response_kind =sub['kind']
            publishedAt=sub['snippet']['publishedAt']
            channel_title =sub['snippet']['title']
            channel_description =sub['snippet']['description']
            channelId = sub['snippet']['resourceId']['channelId']
            sub_channelId = sub['snippet']['channelId']        

            try:
                if response['nextPageToken'] != None:
                    pageToken = response['nextPageToken']
                    df = df.append({'response_kind': response_kind, 'publishedAt':publishedAt, 'channel_title':channel_title,'channel_description':channel_description,'channelId':channelId,'sub_channelId':sub_channelId,'pageToken':pageToken},ignore_index=True)    
            except:
                #save data in pandas df
                df = df.append({'response_kind': response_kind, 'publishedAt':publishedAt, 'channel_title':channel_title,'channelId':channelId,'sub_channelId':sub_channelId,'pageToken':'none'},ignore_index=True)
    
    return df

In [32]:
def get_subs_pagination(df,request,response,youtube):
    while 1:
        try:
            request = youtube.subscriptions().list_next(previous_request=request, previous_response=response)
            response = request.execute()
            df = get_subscriber_ids(df,response)
        
        except:
            break
    
    return df,request,response

In [27]:
dfo = pd.read_csv('strata_unique_commenters_with_1k_of_subscribers.csv',low_memory=False)
df_combo = pd.read_csv('all_ss_comsubs_combined.csv',low_memory=False)

## Main

In [48]:
start = 1218 #after monday morning run
# start = last_user
print(start)

1113


In [49]:
#main
df = pd.DataFrame()

goal = start +300
#get first response

### set up for the wednesday
for channelId in dfo.channelId[start:goal]:

    try:    
        if channelId not in df_combo.sub_channelId.unique(): # check to make sure this is a new sub_channelId
            
            response ,request, youtube = get_subcomms_subscriptions_first_response(channelId)
            #turn first response to df
            df = get_subscriber_ids(df,response)
            print('df length: ' + str(len(df)))

            df,request,response = get_subs_pagination(df=df,response=response ,request=request, youtube=youtube)
            print("user count: " +str(df.sub_channelId.nunique()))
        else:
            pass
    except:
        pass
    
last_user = df.sub_channelId.nunique() + start #create last user variable to use for name

df.to_csv(f'all_ss_comsubs_subscriptions_{start}_{last_user}.csv',index=False) # save as csv
print(f'Saved: all_ss_comsubs_subscriptions_{start}_{last_user}.csv') 

dfall = pd.read_csv('all_ss_comsubs_combined.csv',low_memory=False) #load combined csv
dfall = pd.concat([dfall,df]) # concat to new df

# save newly combined and print the number of unique users
dfall.to_csv('all_ss_comsubs_combined.csv',index=False) ;print(f'total combined Uniques: '+ str(dfall.sub_channelId.nunique())) 

start=last_user #set up start for next time around
print(f'start = {start}')
df.tail(3)

df length: 5
user count: 1
df length: 291
user count: 2
df length: 779
user count: 3
df length: 1063
user count: 4
df length: 1381
user count: 5
df length: 1540
user count: 6
df length: 1588
user count: 7
df length: 1773
user count: 8
df length: 1808
user count: 9
df length: 1854
user count: 10
df length: 1933
user count: 11
df length: 1966
user count: 12
df length: 2065
user count: 13
df length: 2213
user count: 14
df length: 2287
user count: 15
df length: 2535
user count: 16
df length: 2592
user count: 17
df length: 2865
user count: 18
df length: 2877
user count: 19
df length: 2941
user count: 20
df length: 3280
user count: 21
df length: 3533
user count: 22
df length: 4451
user count: 23
df length: 4726
user count: 24
df length: 5139
user count: 25
df length: 5209
user count: 26
df length: 5903
user count: 27
df length: 6130
user count: 28
df length: 6225
user count: 29
df length: 6546
user count: 30
df length: 6689
user count: 31
df length: 7627
user count: 32
df length: 8105
user c

FileNotFoundError: [Errno 2] No such file or directory: 'all_ss_comsubs_combined_.csv'

In [50]:
##########
## can delete now.  thisis in the cell above
dfall = pd.read_csv('all_ss_comsubs_combined.csv',low_memory=False) #load combined csv
dfall = pd.concat([dfall,df]) # concat to new df

# save newly combined and print the number of unique users
dfall.to_csv('all_ss_comsubs_combined.csv',index=False) ;print(f'total combined Uniques: '+ str(dfall.sub_channelId.nunique())) 

start=last_user #set up start for next time around
print(f'start = {start}')
df.tail(3)

total combined Uniques: 1076
start = 1218


,response_kind,publishedAt,channel_title,channel_description,channelId,sub_channelId,pageToken
23932,youtube#subscription,2022-01-19T10:59:31.21189Z,Aishwarya Srinivasan,Hi you! Thanks for stopping by on my channel.\...,UCzd4ZN716evEjtbJERBMTfg,UCPQZDOCaogT1d_OfD167adA,CK8BEAA
23933,youtube#subscription,2011-12-11T12:18:00Z,sciencecomedian,"Earth's Premier Science Comedian, self-proclai...",UCzd6nZNHc0thJsUs-6XmZew,UCPQZDOCaogT1d_OfD167adA,CK8BEAA
23934,youtube#subscription,2021-10-02T16:31:13.558398Z,Siksharthakam,NaN,UCze70Wozmq_2VzhrNbziSgg,UCPQZDOCaogT1d_OfD167adA,none
